In [1]:
import pandas as pd
import numpy as np
import sklearn.datasets

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score,
    mean_absolute_error,
    mean_squared_error,
    classification_report,
    confusion_matrix
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier, LogisticRegression

import requests
import pandas as pd

c:\Users\ignac\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Función para obtener respuesta una vez se le pasa una url o api:

In [2]:
def get_response(url):
    response = requests.get(url)
    return response

Función para obtener los pilotos, con sus escuderías, dado un año:

In [3]:
def obtener_pilotos_año(año):
    url_pilotos = f"http://ergast.com/api/f1/{año}/drivers.json"
    response_pilotos = requests.get(url_pilotos)

    # Verificar que la solicitud fue exitosa
    if response_pilotos.status_code == 200:
        data_pilotos = response_pilotos.json()
        pilotos = data_pilotos['MRData']['DriverTable']['Drivers']
        
        # Diccionario para almacenar el piloto y su escudería
        pilotos_escuderias = {}

        # Iterar sobre cada piloto
        for piloto in pilotos:
            driver_id = piloto['driverId']
            
            # URL para obtener el equipo del piloto específico
            url_constructor = f"http://ergast.com/api/f1/{año}/drivers/{driver_id}/constructors.json"
            response_constructor = requests.get(url_constructor)
            
            if response_constructor.status_code == 200:
                data_constructor = response_constructor.json()
                constructor_name = data_constructor['MRData']['ConstructorTable']['Constructors'][0]['name']
                
                # Guardar el piloto y su equipo en el diccionario
                pilotos_escuderias[driver_id] = constructor_name
            else:
                print(f"Error al obtener la escudería para el piloto {driver_id}")

        # Mostrar resultados
        df_pilotos_escuderias = pd.DataFrame(list(pilotos_escuderias.items()), columns=['driverId', 'escuderia'])
        df_pilotos_escuderias['año']=año
        return df_pilotos_escuderias

    else:
        print("Error al obtener la lista de pilotos:", response_pilotos.status_code)

Código para generar el dataframe con el listado de todos los pitstops desde 2011 hasta 2024

In [4]:
#Lista de años a estudiar:
años = list(range(2011, 2025))

#Columnas del datafram esperado:
df_combined = pd.DataFrame(data=[], columns=["driverId", "duration", "lap", "stop", "time", "año"])


for año in años:
    df_pilotos_escuderias=obtener_pilotos_año(año)
    url = "https://ergast.com/api/f1/"+str(año)+".json"
    response=get_response(url)

    # Si la petición es exitosa (código 200)
    if response.status_code == 200:
        # Obtener los datos en formato JSON
        data = response.json()
        #Obtener el número de carreras por temporada
        num_carreras= int(data['MRData']['RaceTable']['Races'][-1]['round'])
        rounds = list(range(1, (num_carreras+1)))
        for round in rounds:
            url="https://ergast.com/api/f1/"+str(año)+"/"+str(round)+"/pitstops.json"
            response=get_response(url)
            if response.status_code == 200:
                # Obtener los datos en formato JSON
                data = response.json()
                try:    
                    #pitstops de los pilotos en un año:
                    pitstop_año_circuito = data['MRData']['RaceTable']['Races'][0]['PitStops']
                    #pasar a dataframe:
                    df_sin_scuderia=pd.DataFrame(pitstop_año_circuito)
                    #se hace un merge para obtener las escuderías:
                    df_resultado = pd.merge(df_sin_scuderia, df_pilotos_escuderias, on='driverId', how='inner')
                    #se va añadiendo al dataframe final:
                    df_combined = df_combined.append(df_resultado, ignore_index=True)

                except IndexError:
                    print (f"Error in round {round} year: {año}")
            
df_combined

c:\Users\ignac\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Error in round 12 year: 2021
Error in round 22 year: 2024
Error in round 23 year: 2024
Error in round 24 year: 2024


,año,driverId,duration,escuderia,lap,stop,time
0,2011,alguersuari,26.898,Toro Rosso,1,1,17:05:23
1,2011,alguersuari,24.463,Toro Rosso,17,2,17:31:06
2,2011,michael_schumacher,25.021,Mercedes,1,1,17:05:52
3,2011,michael_schumacher,23.988,Mercedes,17,2,17:32:08
4,2011,webber,23.426,Red Bull,11,1,17:20:48
5,2011,webber,22.520,Red Bull,26,2,17:44:29
6,2011,alonso,23.251,Ferrari,12,1,17:22:34
7,2011,alonso,24.733,Ferrari,27,2,17:46:04
8,2011,massa,23.842,Ferrari,13,1,17:24:10
9,2011,barrichello,23.643,Williams,13,1,17:24:29


Pasar y almacenar dataframe a un fichero Excel

In [7]:
# Save df_combined to an Excel file
df_combined.to_excel("storage/Historical Pitstops.xlsx", index=False)


Agrupar el dataframe para mostrar la evolución anual de pitstops por equipo

In [8]:
#obtener df_combined siendo un dataframe que se genera a partir del excel 'Historical Pitstops.xlsx'
df_combined = pd.read_excel('storage/Historical Pitstops.xlsx')
#agrupame el df_combined por año y por escuderia
df_combined['duration'] = pd.to_numeric(df_combined['duration'], errors='coerce')
# Now, group by 'año' and 'escuderia' and calculate the mean 'duration'
grouped = df_combined.groupby(['año', 'escuderia'])
avg_duration = grouped['duration'].mean().reset_index()

Pasar y almacenar dataframe agrupado a un fichero Excel

In [9]:
avg_duration.to_excel("storage/Historical Pitstops Grouped.xlsx", index=False)
